<a href="https://colab.research.google.com/github/Quikks1lver/WayFinder/blob/main/NodeXYExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import xml.etree.ElementTree as et
import json
import time

In [ ]:
xml = files.upload()

Saving mapnew.osm to mapnew.osm


In [ ]:
# Load in the osm XML file and get the root

mytree = et.parse("mapnew.osm")

myroot = mytree.getroot()

print(myroot)

<Element 'osm' at 0x7ff5d5373098>


In [ ]:
# Initialize output dictionary/JS object. Holds node id references : node information (id, x, y, list of ways the node is a part of, etc.)
outputMap: dict = {}

# Map holding node information -- node id : w/lat, long
nodeInfoMap: dict = {}

# Stores temporarily the reference id for each node
nodeId: int = 0

# Stores temporarily long/lat for each node
longX: float = 0.0
latY: float = 0.0

# Map to hold lat/long for nodes, temporarily
nodeGeolocation = {}

# Map to hold way info for the nodes
wayInfoForNodeMap: dict = {}

In [ ]:
# create a hash map to store way : node info
for node in myroot.findall("node"):
  longX = node.attrib["lon"]
  latY = node.attrib["lat"]

  nodeGeolocation = {}
  nodeGeolocation["longX"] = longX
  nodeGeolocation["latY"] = latY

  nodeInfoMap[int(node.attrib["id"])] = nodeGeolocation

In [ ]:
print(nodeInfoMap)

{99457766: {'longX': '-81.2076978', 'latY': '28.6118547'}, 99457791: {'longX': '-81.1745392', 'latY': '28.6120393'}, 99466414: {'longX': '-81.2090441', 'latY': '28.5960278'}, 99470893: {'longX': '-81.1770073', 'latY': '28.6058723'}, 99472146: {'longX': '-81.1783537', 'latY': '28.5803939'}, 99477714: {'longX': '-81.1835598', 'latY': '28.6011167'}, 99480102: {'longX': '-81.1783679', 'latY': '28.5811407'}, 99489389: {'longX': '-81.2269810', 'latY': '28.5902818'}, 99493703: {'longX': '-81.2202729', 'latY': '28.5996148'}, 99505045: {'longX': '-81.1783854', 'latY': '28.5854525'}, 99507439: {'longX': '-81.1783966', 'latY': '28.6072252'}, 99516656: {'longX': '-81.1740889', 'latY': '28.6025792'}, 99520044: {'longX': '-81.2002384', 'latY': '28.5948268'}, 99520984: {'longX': '-81.2090933', 'latY': '28.6004275'}, 99520987: {'longX': '-81.2099320', 'latY': '28.6006394'}, 99533762: {'longX': '-81.1816287', 'latY': '28.6027110'}, 99535539: {'longX': '-81.2114612', 'latY': '28.5919605'}, 99535542: {'l

In [ ]:
count: int = 0
DEBUGGING: bool = True

# for timing the code
t0 = time.time()

# loop through all nodes
for way in myroot.findall("way"):
  
  wayInfoForNodeMap = {}

  if count > 50 and DEBUGGING:
    break

  try:    
    
    # populate id for way in wayinfo map
    wayInfoForNodeMap["wayId"] = int(way.attrib["id"])

    # populate name for way in the wayinfo map
    childCount: int = 0
    for childTag in way.findall("tag"):
      if childCount == 1:
        wayInfoForNodeMap["streetName"] = childTag.attrib["v"]
      childCount += 1

    # draw relationships between ways <=> nodes to add into output map
    for node in way.findall("nd"):
      nodeId = int(node.attrib["ref"])
      
      # new node
      if nodeId not in outputMap.keys():
        # populate dict values for output map
        longX = nodeInfoMap.get(nodeId)["longX"]
        latY = nodeInfoMap.get(nodeId)["latY"]
        tempWayInfo = []
        tempWayInfo.append(wayInfoForNodeMap)

        outputMap[nodeId] = {"id": nodeId, "longX": longX, "latY": latY, "waysInfo": tempWayInfo}

      # duplicate node, so add additional way to waysInfo list
      else:
        outputMap[nodeId]["waysInfo"].append(wayInfoForNodeMap)
        
        if DEBUGGING:
          print("duplicate node on " + str(nodeId))

      if DEBUGGING:
        print(outputMap)

    if DEBUGGING:
      print(wayInfoForNodeMap, end="\n\n")

    count += 1

  except:
    continue

t1 = time.time()

print("================================================")
print(outputMap)
print(f"Code took {t1 - t0} to run")
print("================================================")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
outputData

In [ ]:
# the json file where the output must be stored 

with open("nodeInfo.json", "w") as outputFile:
  json.dump(outputMap, outputFile)